# Introduction

This notebook is the workings for part (c) of the [Study Question 1.5.2](study_question_1.5.2.ipynb#(c)) notebook.

# Packages

In [1]:
using JuMP, Ipopt

# Solution

Define the problem:

In [2]:
model = Model(Ipopt.Optimizer)

# all the parameters are probabilities
@variable(model, 0 <= p₁ <= 1)
@variable(model, 0 <= p₂ <= 1)
@variable(model, 0 <= p₃ <= 1)
@variable(model, 0 <= p₄ <= 1)

@variable(model, 0.5 <= q₁ <= 1) # very likely to take treatment if no syndrome
@variable(model, 0 <= q₂ <= 0.5) # less likely to take treament if have syndrome

@variable(model, 0 <= r <= 0.2) # assume only small fraction of population have the syndrome

# we want the subpopulation to show bad results
@constraint(model, p₄ - p₃ <= -0.1) # b(i) constraint to see big negative effect
@constraint(model, p₂ - p₁ <= -0.1) # b(ii) constraint to see big negative effect
# @constraint(model, p₄ - p₃  == p₂ - p₁) # b(ii) constraint


# objective function is the b(iii)
# we want to maximize this
obj_fn(p₁, p₂, p₃, p₄, q₁, q₂, r) = begin
    term1 = (q₁ * p₂ * (1-r) + q₂ * p₄ * r)/(-q₁ * r + q₂ * r + q₁)
    term2 = ((1 - q₁) * p₁ * (1-r) + (1-q₂) * p₃ * r)/(q₁ * r - q₂ * r - q₁ + 1)
    
    term1 - term2
end

register(model, :obj_fn, 7, obj_fn, autodiff=true)
@NLobjective(model, Max, obj_fn(p₁, p₂, p₃, p₄, q₁, q₂, r))

model

A JuMP Model
Maximization problem with:
Variables: 7
Objective function type: Nonlinear
`GenericAffExpr{Float64,VariableRef}`-in-`MathOptInterface.LessThan{Float64}`: 2 constraints
`VariableRef`-in-`MathOptInterface.GreaterThan{Float64}`: 7 constraints
`VariableRef`-in-`MathOptInterface.LessThan{Float64}`: 7 constraints
Model mode: AUTOMATIC
CachingOptimizer state: EMPTY_OPTIMIZER
Solver name: Ipopt
Names registered in the model: p₁, p₂, p₃, p₄, q₁, q₂, r

Find the solution (if any):

In [ ]:
optimize!(model)

Show the solution:

In [ ]:
@show objective_value(model)

@show value(p₁)
@show value(p₂)
@show value(p₃)
@show value(p₄)
@show value(q₁)
@show value(q₂)
@show value(r)
;